In [14]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

def feature_reduction(df, config):
    """
    Perform feature reduction based on the configuration provided in the JSON.
    :param df: The input DataFrame
    :param config: The JSON configuration for feature reduction
    :return: A DataFrame with reduced features
    """
    feature_reduction_config = config["feature_reduction"]
    method = feature_reduction_config["feature_reduction_method"]
    
    target_column = config["target"]["target"]

    # Ensure the target column exists in the DataFrame
    if target_column not in df.columns:
        raise ValueError(f"Target column '{target_column}' not found in DataFrame")

    # Handle non-numeric columns by encoding them
    for column in df.select_dtypes(include=['object']).columns:
        # Encoding categorical columns into numeric values
        encoder = LabelEncoder()
        df[column] = encoder.fit_transform(df[column])

    # No Reduction
    if feature_reduction_config.get("No reduction", {}).get("is_selected", False):
        print(f"Performing 'No reduction'. Keeping all features.")
        return df
    
    # Correlation with Target
    elif feature_reduction_config.get("Correlation with target", {}).get("is_selected", False):
        print(f"Performing 'Correlation with target'. Selecting top {feature_reduction_config['Correlation with target']['num_of_features_to_keep']} features.")
        target = df[target_column]
        correlations = df.corrwith(target).abs()
        selected_features = correlations.nlargest(feature_reduction_config['Correlation with target']['num_of_features_to_keep']).index
        return df[selected_features]
    
    # Tree-based
    elif feature_reduction_config.get("Tree-based", {}).get("is_selected", False):
        print(f"Performing 'Tree-based'. Selecting top {feature_reduction_config['Tree-based']['num_of_features_to_keep']} features.")
        X = df.drop(columns=[target_column])
        y = df[target_column]
        
        model = RandomForestRegressor(n_estimators=feature_reduction_config['Tree-based'].get('num_of_trees', 100),
                                      max_depth=feature_reduction_config['Tree-based'].get('depth_of_trees', 5))
        model.fit(X, y)
        
        feature_importance = pd.Series(model.feature_importances_, index=X.columns).nlargest(feature_reduction_config['Tree-based']['num_of_features_to_keep'])
        selected_features = feature_importance.index
        return df[selected_features]
    
    # Principal Component Analysis (PCA)
    elif feature_reduction_config.get("Principal Component Analysis", {}).get("is_selected", False):
        print(f"Performing 'Principal Component Analysis'. Selecting top {feature_reduction_config['Principal Component Analysis']['num_of_features_to_keep']} components.")
        X = df.drop(columns=[target_column])
        pca = PCA(n_components=feature_reduction_config['Principal Component Analysis']['num_of_features_to_keep'])
        principal_components = pca.fit_transform(X)
        
        pca_df = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(feature_reduction_config['Principal Component Analysis']['num_of_features_to_keep'])])
        return pca_df
    
    else:
        print(f"Unknown feature reduction method: {method}")
        return df

# JSON configuration 
json_config = {
    "design_state_data": {
        "session_info": {
            "project_id": "1",
            "experiment_id": "kkkk-11",
            "dataset": "iris_modified.csv",
            "session_name": "test",
            "session_description": "test"
        },
        "target": {
            "prediction_type": "Regression",
            "target": "petal_width",
            "type": "regression",
            "partitioning": True
        },
        "feature_reduction": {
            "feature_reduction_method": "Tree-based",
            "No reduction": {
                "is_selected": False,
                "num_of_features_to_keep": 5
            },
            "Correlation with target": {
                "is_selected": False,
                "num_of_features_to_keep": 0
            },
            "Tree-based": {
                "is_selected": True,
                "num_of_features_to_keep": 4,
                "depth_of_trees": 6,
                "num_of_trees": 5
            },
            "Principal Component Analysis": {
                "is_selected": False,
                "num_of_features_to_keep": 0
            }
        }
    }
}

file_path = "C:/Users/Elakkiya/Downloads/iris_modified.csv"
df = pd.read_csv(file_path)

# Extract the relevant configuration for feature reduction
feature_reduction_config = json_config["design_state_data"]

# Perform feature reduction based on the configuration
reduced_df = feature_reduction(df, feature_reduction_config)
print(reduced_df)


Performing 'Tree-based'. Selecting top 4 features.
     petal_length  species  sepal_width  sepal_length
0             1.4        0          3.5           5.1
1             1.4        0          3.0           4.9
2             1.3        0          3.2           4.7
3             1.5        0          3.1           4.6
4             1.4        0          3.6           5.0
..            ...      ...          ...           ...
145           5.2        2          3.0           6.7
146           5.0        2          2.5           6.3
147           5.2        2          3.0           6.5
148           5.4        2          3.4           6.2
149           5.1        2          3.0           5.9

[150 rows x 4 columns]


In [21]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Function to initialize models based on the prediction type and JSON configuration
def initialize_model(json_config):
    prediction_type = json_config["design_state_data"]["target"]["prediction_type"]
    model_config = json_config["design_state_data"]["algorithms"]

    models = []

    # Check for "Regression" or "Classification"
    if prediction_type == "Regression":
        print("Selected prediction type: Regression")

        # Select and initialize regression models based on JSON configuration
        if "LinearRegression" in model_config and model_config["LinearRegression"].get("is_selected", False):
            model = LinearRegression()
            models.append(model)
            print("LinearRegression selected.")

        if "RandomForestRegressor" in model_config and model_config["RandomForestRegressor"].get("is_selected", False):
            model = RandomForestRegressor(
                n_estimators=model_config["RandomForestRegressor"].get("n_estimators", 100),
                max_depth=model_config["RandomForestRegressor"].get("max_depth", None),
                min_samples_split=model_config["RandomForestRegressor"].get("min_samples_split", 2)
            )
            models.append(model)
            print("RandomForestRegressor selected.")

        if "SVR" in model_config and model_config["SVR"].get("is_selected", False):
            model = SVR(
                C=model_config["SVR"].get("C", 1.0),
                kernel=model_config["SVR"].get("kernel", 'rbf')
            )
            models.append(model)
            print("SVR selected.")

        if "KNeighborsRegressor" in model_config and model_config["KNeighborsRegressor"].get("is_selected", False):
            model = KNeighborsRegressor(
                n_neighbors=model_config["KNeighborsRegressor"].get("n_neighbors", 5)
            )
            models.append(model)
            print("KNeighborsRegressor selected.")

    elif prediction_type == "Classification":
        print("Selected prediction type: Classification")

        # Select and initialize classification models based on JSON configuration
        if "LogisticRegression" in model_config and model_config["LogisticRegression"].get("is_selected", False):
            model = LogisticRegression(
                solver=model_config["LogisticRegression"].get("solver", "lbfgs"),
                max_iter=model_config["LogisticRegression"].get("max_iter", 100),
                C=model_config["LogisticRegression"].get("min_regparam", 0.5),
                penalty=model_config["LogisticRegression"].get("penalty", "l2")
            )
            models.append(model)
            print("LogisticRegression selected.")

        if "RandomForestClassifier" in model_config and model_config["RandomForestClassifier"].get("is_selected", False):
            model = RandomForestClassifier(
                n_estimators=model_config["RandomForestClassifier"].get("n_estimators", 100),
                max_depth=model_config["RandomForestClassifier"].get("max_depth", None)
            )
            models.append(model)
            print("RandomForestClassifier selected.")

        if "SVC" in model_config and model_config["SVC"].get("is_selected", False):
            model = SVC(
                C=model_config["SVC"].get("C", 1.0),
                kernel=model_config["SVC"].get("kernel", 'rbf')
            )
            models.append(model)
            print("SVC selected.")

        if "KNeighborsClassifier" in model_config and model_config["KNeighborsClassifier"].get("is_selected", False):
            model = KNeighborsClassifier(
                n_neighbors=model_config["KNeighborsClassifier"].get("n_neighbors", 5)
            )
            models.append(model)
            print("KNeighborsClassifier selected.")
    
    else:
        raise ValueError(f"Unknown prediction type: {prediction_type}")

    return models

json_path = "C:/Users/Elakkiya/Downloads/algoparams_from_ui.json"
with open(json_path, 'r') as file:
    json_config = json.load(file)

dataset_path = "C:/Users/Elakkiya/Downloads/iris_modified.csv"
data = pd.read_csv(dataset_path)

# Extract features and target based on the JSON config
target_column = json_config["design_state_data"]["target"]["target"]
X = data.drop(columns=[target_column])
y = data[target_column]

# Encode categorical features if necessary
categorical_columns = X.select_dtypes(include=['object']).columns
if not categorical_columns.empty:
    X = pd.get_dummies(X, columns=categorical_columns)

# Encode target column if it's a classification problem
prediction_type = json_config["design_state_data"]["target"]["prediction_type"]
if prediction_type == "Classification":
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models based on JSON configuration
models = initialize_model(json_config)

# Train and evaluate each selected model
for model in models:
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluation based on prediction type
    if prediction_type == "Regression":
        # Evaluate using Mean Squared Error
        mse = mean_squared_error(y_test, y_pred)
        print(f"Model {model} MSE: {mse}")
    elif prediction_type == "Classification":
        # Evaluate using Accuracy
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model {model} Accuracy: {accuracy}")
    else:
        print("Unknown prediction type")


Selected prediction type: Regression
RandomForestRegressor selected.
Model RandomForestRegressor(max_depth=25) MSE: 0.038914910393046016


In [6]:
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import LabelEncoder

json_path = "C:/Users/Elakkiya/Downloads/algoparams_from_ui.json"
with open(json_path, 'r') as f:
    algoparams = json.load(f)

csv_path = "C:/Users/Elakkiya/Downloads/iris_modified.csv"
data = pd.read_csv(csv_path)

# Extract design state data and hyperparameters
design_state_data = algoparams['design_state_data']
hyperparameters = design_state_data['hyperparameters']

# Set default values for test_size and random_state if not provided
test_size = hyperparameters.get('test_size', 0.2)
random_state = hyperparameters.get('random_state', 0)

# Extract target column and prepare the data
target_info = design_state_data['target']
target_column = target_info['target']
X = data.drop(target_column, axis=1)
y = data[target_column]

# If this is a classification problem, encode the target labels
if target_info['prediction_type'] == 'Classification':
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
# Identify categorical columns in X
categorical_columns = X.select_dtypes(include=['object', 'category']).columns

# Apply label encoding to categorical columns in X
for col in categorical_columns:
    label_encoder = LabelEncoder()
    X[col] = label_encoder.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Define models and parameter grids
models = {}
param_grids = {}

# Loop through the algorithms and their parameters
for model_key, model_info in design_state_data['algorithms'].items():
    if model_info['is_selected']:
        model_name = model_info['model_name']
        
        if model_name == 'Random Forest Regressor':
            models[model_name] = RandomForestRegressor(random_state=random_state)
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'Random Forest Classifier':
            models[model_name] = RandomForestClassifier(random_state=random_state)
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'Gradient Boosting Regressor':
            models[model_name] = GradientBoostingRegressor(random_state=random_state)
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'Gradient Boosting Classifier':
            models[model_name] = GradientBoostingClassifier(random_state=random_state)
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'Linear Regression':
            models[model_name] = LinearRegression()
            param_grids[model_name] = {}
        elif model_name == 'Logistic Regression':
            models[model_name] = LogisticRegression(max_iter=1000, random_state=random_state)
            param_grids[model_name] = {}
        elif model_name == 'Ridge Regression':
            models[model_name] = Ridge()
            param_grids[model_name] = {
                'alpha': [0.1, 1.0, 10.0]
            }
        elif model_name == 'Lasso Regression':
            models[model_name] = Lasso()
            param_grids[model_name] = {
                'alpha': [0.1, 1.0, 10.0]
            }
        elif model_name == 'ElasticNet Regression':
            models[model_name] = ElasticNet()
            param_grids[model_name] = {
                'alpha': [0.1, 1.0, 10.0],
                'l1_ratio': [0.1, 0.5, 0.9]
            }
        elif model_name == 'Decision Tree Regressor':
            models[model_name] = DecisionTreeRegressor(random_state=random_state)
            param_grids[model_name] = {
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'Decision Tree Classifier':
            models[model_name] = DecisionTreeClassifier(random_state=random_state)
            param_grids[model_name] = {
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'SVR':
            models[model_name] = SVR()
            param_grids[model_name] = {
                'C': [0.1, 1, 10],
                'kernel': ['linear', 'rbf']
            }
        elif model_name == 'SVC':
            models[model_name] = SVC(random_state=random_state)
            param_grids[model_name] = {
                'C': [0.1, 1, 10],
                'kernel': ['linear', 'rbf']
            }
        elif model_name == 'SGD Regressor':
            models[model_name] = SGDRegressor()
            param_grids[model_name] = {
                'loss': ['squared_loss', 'huber'],
                'alpha': [0.1, 0.01, 0.001]
            }
        elif model_name == 'SGD Classifier':
            models[model_name] = SGDClassifier()
            param_grids[model_name] = {
                'loss': ['hinge', 'log', 'modified_huber'],
                'alpha': [0.1, 0.01, 0.001]
            }
        elif model_name == 'KNN Regressor':
            models[model_name] = KNeighborsRegressor()
            param_grids[model_name] = {
                'n_neighbors': [3, 5, 7]
            }
        elif model_name == 'KNN Classifier':
            models[model_name] = KNeighborsClassifier()
            param_grids[model_name] = {
                'n_neighbors': [3, 5, 7]
            }
        elif model_name == 'MLP Regressor':
            models[model_name] = MLPRegressor(max_iter=1000, random_state=random_state)
            param_grids[model_name] = {
                'hidden_layer_sizes': [(50,), (100,), (50, 50)],
                'activation': ['tanh', 'relu']
            }
        elif model_name == 'MLP Classifier':
            models[model_name] = MLPClassifier(max_iter=1000, random_state=random_state)
            param_grids[model_name] = {
                'hidden_layer_sizes': [(50,), (100,), (50, 50)],
                'activation': ['tanh', 'relu']
            }
        elif model_name == 'XGBoost Regressor':
            models[model_name] = XGBRegressor(random_state=random_state)
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }
        elif model_name == 'XGB Classifier':
            models[model_name] = XGBClassifier(random_state=hyperparameters['random_state'])
            param_grids[model_name] = {
                'n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [model_info['min_depth'], model_info['max_depth']]
            }

# Set default values for Time-based K-fold if not defined
num_folds = 5  # Default value
split_ratio = 0.8  # Default value

# Apply Time-based K-fold (with overlap) cross-validation strategy
if hyperparameters.get('cross_validation_strategy') == 'Time-based K-fold(with overlap)':
    time_based_k_fold = hyperparameters.get('Time-based K-fold(with overlap)', {})
    num_folds = time_based_k_fold.get('num_of_folds', 0)
    split_ratio = time_based_k_fold.get('split_ratio', 0)

# Create TimeSeriesSplit with the required number of folds
tscv = TimeSeriesSplit(n_splits=num_folds)

# Perform cross-validation for each selected model
for model_name, model in models.items():
    print(f"Training model: {model_name}")
    
    grid_search = GridSearchCV(model, param_grids[model_name], cv=tscv)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
    
    # Evaluate model performance on test data
    y_pred = best_model.predict(X_test)
    if target_info['prediction_type'] == 'Regression':
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Mean Squared Error for {model_name}: {mse}")
        print(f"R^2 Score for {model_name}: {r2}")
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f"Accuracy for {model_name}: {accuracy}")
        print(f"Precision for {model_name}: {precision}")
        print(f"Recall for {model_name}: {recall}")
        print(f"F1 Score for {model_name}: {f1}")


Training model: Random Forest Regressor
Best Parameters for Random Forest Regressor: {'max_depth': 20, 'n_estimators': 20}
Mean Squared Error for Random Forest Regressor: 0.026058645370370344
R^2 Score for Random Forest Regressor: 0.9462634478202426
